## Using Ridge and Lasso Regression 

Nikolas Hermanowycz \

In [112]:
!pip install sklearn

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as opt
from scipy import stats
from random import random
from mip import Model, minimize, BINARY, INTEGER, CONTINUOUS, xsum 

In [21]:
df = pd.read_csv('buan685_hw4_absenteeism.csv')#Age, LengthService, Gender, Division, and BusinessUnit
df = df.drop(['EmployeeNumber','Surname','GivenName', 'City', 'JobTitle', 'DepartmentName', 'StoreLocation', 'Division'], axis = 1)#'GivenName', 'City', 'JobTitle', 'DepartmentName', 'StoreLocation', 'Division'
df

,Gender,Age,LengthService,AbsentHours,BusinessUnit
0,F,32.028816,6.018478,36.577306,Stores
1,M,40.320902,5.532445,30.165072,Stores
2,M,48.822047,4.389973,83.807798,Stores
3,F,44.599357,3.081736,70.020165,Stores
4,M,35.697876,3.619091,0.000000,Stores
...,...,...,...,...,...
8331,F,46.057544,4.838288,93.665111,Stores
8332,M,34.455490,2.427274,0.000000,Stores
8333,F,58.347160,4.009393,176.356940,Stores
8334,F,43.340616,6.154837,60.321917,Stores


In [22]:
 df.isna().sum() #shows NA or missing values


Gender           0
Age              0
LengthService    0
AbsentHours      0
BusinessUnit     0
dtype: int64

In [23]:
## split data into training, validation and test

from sklearn.model_selection import train_test_split

In [37]:
#, 'Age','LengthService','AbsentHours', 'BusinessUnit'
X = df.drop('AbsentHours', axis = 1)
y = df['AbsentHours']

In [38]:
X = pd.get_dummies(X)
#60:20:20 split

In [39]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2)

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.25 )

In [41]:
len(X_train), len(X_valid), len(X_test) 

(5001, 1667, 1668)

In [42]:
len(y_train), len(y_valid), len(y_test)

(5001, 1667, 1668)

# Linear Regression 

In [43]:
X_train.values

array([[46.55655861,  7.73509005,  1.        ,  0.        ,  0.        ,
         1.        ],
       [35.02588628,  0.77436544,  1.        ,  0.        ,  0.        ,
         1.        ],
       [31.98807499,  4.51595403,  1.        ,  0.        ,  0.        ,
         1.        ],
       ...,
       [42.3939824 ,  4.97584095,  1.        ,  0.        ,  0.        ,
         1.        ],
       [41.73571778,  5.83771805,  0.        ,  1.        ,  0.        ,
         1.        ],
       [24.25596785,  5.21461616,  1.        ,  0.        ,  0.        ,
         1.        ]])

In [44]:
X_train['constant'] = 1

<ipython-input-44-0846bf6f4714>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['constant'] = 1


In [46]:
x = X_train.values

In [48]:
x.T

array([[46.55655861, 35.02588628, 31.98807499, ..., 42.3939824 ,
        41.73571778, 24.25596785],
       [ 7.73509005,  0.77436544,  4.51595403, ...,  4.97584095,
         5.83771805,  5.21461616],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ]])

In [50]:
# matrix multiplica
x.T.dot(x)

array([[9.29217649e+06, 1.00512983e+06, 1.01496696e+05, 1.08204207e+05,
        4.19604194e+03, 2.05504862e+05, 2.09700904e+05],
       [1.00512983e+06, 1.40352979e+05, 1.18093625e+04, 1.20335818e+04,
        1.30916723e+03, 2.25337771e+04, 2.38429443e+04],
       [1.01496696e+05, 1.18093625e+04, 2.46500000e+03, 0.00000000e+00,
        4.70000000e+01, 2.41800000e+03, 2.46500000e+03],
       [1.08204207e+05, 1.20335818e+04, 0.00000000e+00, 2.53600000e+03,
        4.40000000e+01, 2.49200000e+03, 2.53600000e+03],
       [4.19604194e+03, 1.30916723e+03, 4.70000000e+01, 4.40000000e+01,
        9.10000000e+01, 0.00000000e+00, 9.10000000e+01],
       [2.05504862e+05, 2.25337771e+04, 2.41800000e+03, 2.49200000e+03,
        0.00000000e+00, 4.91000000e+03, 4.91000000e+03],
       [2.09700904e+05, 2.38429443e+04, 2.46500000e+03, 2.53600000e+03,
        9.10000000e+01, 4.91000000e+03, 5.00100000e+03]])

In [56]:
a = np.linalg.inv(x.T.dot(x)).dot((x.T.dot(y_train)))
a

array([    4.67611271,   -11.17818972, -1920.        , -2176.        ,
         896.        ,   768.        ,  1280.        ])

In [53]:
X_valid['constant']= 1

<ipython-input-53-b85e085df520>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid['constant']= 1


In [60]:
y_valid_pred = X_valid.values.dot(a)
y_valid_pred

array([297.11984367,  58.25211072,  25.91025309, ..., 280.88811476,
        69.92434621, 262.38135922])

In [63]:
a = np.linalg.solve(x.T.dot(x), x.T.dot(y_train))
a

array([ 4.19350454e+00, -8.11180918e-01, -1.89547126e+03, -1.91207597e+03,
        7.51002741e+02,  7.72549398e+02,  1.02083855e+03])

In [65]:
y_valid_pred = X_valid.values.dot(a)

array([105.11486131,  57.55488795,  69.58506161, ...,  63.87224293,
        90.5004462 ,  67.29983099])

In [67]:
X_test['constant']=1

<ipython-input-67-e8d3d75fa1ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['constant']=1


In [68]:
y_test_pred = X_test.values.dot(a)

In [70]:
RMSE = np.mean((y_test_pred-y_test)**2)**0.5

In [73]:
results = {'linear':RMSE}
results

{'linear': 25.58251609409401}

# Linear Regression Sklarn package

In [77]:
X_train = X_train.drop('constant', axis= 1)
X_valid = X_valid.drop('constant', axis= 1)
X_test = X_test.drop('constant', axis= 1)

In [78]:
from sklearn.linear_model import LinearRegression 

In [81]:
lr = LinearRegression()

In [82]:
lr.fit(X_train, y_train)

LinearRegression()

In [83]:
lr.coef_

array([  4.19350454,  -0.81118092,   8.30235604,  -8.30235604,
       -10.77332837,  10.77332837])

# Ridge Regression

In [88]:
from sklearn.linear_model import Ridge

In [91]:
ridge = Ridge()

In [92]:
ridge.fit(X_train, y_train)

Ridge()

In [93]:
y_valid_ridge = ridge.predict(X_valid)

In [94]:
RMSE = np.mean((y_valid_ridge-y_valid)**2)**0.5

In [95]:
RMSE

25.295946726232184

# Lasso Regression 

In [98]:
from sklearn.linear_model import Lasso

In [99]:
lasso = Lasso()

In [100]:
lasso.fit(X_train, y_train)

Lasso()

In [101]:
y_valid_lasso = lasso.predict(X_valid)

In [103]:
RMSE = np.mean((y_valid_lasso-y_valid)**2)**0.5
RMSE

25.579797134451233

# Best Model (Validation Set) 

In [106]:
regularizations = [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000, 10000, 1e5, 1e6, 1e7]

In [107]:
ridge_RMSEs = []

In [110]:
for reg in regularizations: 
    ridge = Ridge(alpha=reg)
    ridge.fit(X_train, y_train)
    y_valid_pred - ridge.predict(X_valid)
    RMSE = np.mean((y_valid_pred-y_valid)**2)**0.5
    ridge_RMSEs.append([reg,RMSE])

In [111]:
ridge_RMSEs

[[0.001, 25.295425705063586],
 [0.01, 25.295425705063586],
 [0.1, 25.295425705063586],
 [1, 25.295425705063586],
 [10, 25.295425705063586],
 [100, 25.295425705063586],
 [1000, 25.295425705063586],
 [10000, 25.295425705063586],
 [100000.0, 25.295425705063586],
 [1000000.0, 25.295425705063586],
 [10000000.0, 25.295425705063586]]

In [117]:
lasso_RMSEs = []

In [118]:
for reg in regularizations: 
    lasso = Lasso(alpha=reg)
    lasso.fit(X_train, y_train)
    y_valid_pred = lasso.predict(X_valid)
    RMSE = np.mean((y_valid_pred-y_valid)**2)**0.5
    lasso_RMSEs.append([reg,RMSE])

In [120]:
lasso_RMSEs

[[0.001, 25.29566830430723],
 [0.01, 25.29809058115018],
 [0.1, 25.341273745968117],
 [1, 25.579797134451233],
 [10, 27.19982017959001],
 [100, 28.887402527788787],
 [1000, 48.54019795460301],
 [10000, 48.54019795460301],
 [100000.0, 48.54019795460301],
 [1000000.0, 48.54019795460301],
 [10000000.0, 48.54019795460301]]

# Retrain

In [125]:
lasso = Lasso(alpha=0.001)

In [126]:
lasso.fit(X_train_valid, y_train_valid)

Lasso(alpha=0.001)

In [128]:
y_test_lasso = lasso.predict(X_test)

In [129]:
np.mean((y_test_lasso-y_test)**2)**0.5

25.574607836945034

In [133]:
print('Ridge:25.295946726232184')
print('Lasso:25.579797134451233')
print('Least:25.574607836945034')

Ridge:25.295946726232184
Lasso:25.579797134451233
Least:25.574607836945034
